In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import pandas as pd
import numpy as np
%aimport Variant_analysis_helper_functions

import subprocess
from scipy.stats import chisquare
import os
import protfasta
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
d_domains = "../outputs/mutations/domains_expanded_iWES_v2_variants_snv_classified/"
files = os.listdir(d_domains)
ENST_codes = [f.replace(".bed", "") for f in files]

uniprotID_ENST_mapping = pd.read_csv("../../data/SFARI_TFs_with_ENST.csv")
uniprotID_ENST_mapping = uniprotID_ENST_mapping[["uniprotID", "ENST"]]
uniprotID_ENST_mapping["ENST"] = uniprotID_ENST_mapping["ENST"].str.split(".").str[0]
uniprotID_ENST_mapping_dict= dict(zip(uniprotID_ENST_mapping["ENST"], uniprotID_ENST_mapping["uniprotID"]))
uniprotID_ENST_mapping_dict['ENST00000434704'] = 'O60479'

gnomad = pd.read_csv("../raw_files/gnomad.bed", sep = "\t", header = None)
# gnomad = gnomad[[0, 1, 2, 3, 4]]
gnomad = gnomad.rename(columns = {5 : "gnomad_allele_freq"})
for col_num in [1, 2, "gnomad_allele_freq"]:
    gnomad[col_num] = gnomad[col_num].astype(float)
gnomad[0] = gnomad[0].astype(str)

SFARI_TFs = pd.read_csv("../../data/SFARI_TFs_with_ENST.csv")

In [18]:
gnomad

,0,1,2,3,4,gnomad_allele_freq
0,2,207081140.0,207081141.0,T,C,3.216086e-06
1,2,207081144.0,207081145.0,G,C,7.777610e-07
2,2,207081145.0,207081146.0,C,G,7.714145e-07
3,2,207081146.0,207081147.0,C,T,1.529487e-06
4,2,207081147.0,207081148.0,C,T,5.777024e-06
...,...,...,...,...,...,...
107567,19,13094705.0,13094706.0,G,A,7.334088e-07
107568,19,13094708.0,13094709.0,G,T,1.997268e-06
107569,19,13094710.0,13094711.0,A,G,1.250769e-06
107570,19,13094717.0,13094718.0,A,G,3.763805e-05


In [ ]:
def return_iwes_gnomad_overlap(region_type, ENST, gnomad):
    domain_output = pd.read_csv("../outputs/mutations/" + region_type + "_expanded_iWES_v2_variants_snv_classified/" + ENST + ".bed", sep = "\t", header = None)

    if region_type == "domains":
        domain_output = domain_output[domain_output[22] == "No-Syn"]
        domain_output = domain_output[[0, 1, 2, 17, 18, 19, 3]]
        domain_output = domain_output.rename(columns = {17: 3, 18: 4, 19: "iWES_allele_freq", 3: "domain_type"})
    else:
        domain_output = domain_output[domain_output[14] == "No-Syn"]
        domain_output = domain_output[[0, 1, 2, 9, 10, 11]]
        domain_output = domain_output.rename(columns = {9: 3, 10: 4, 11: "iWES_allele_freq"})
        domain_output["domain_type"] = "cds"
    for col_num in [1, 2, "iWES_allele_freq"]:
        domain_output[col_num] = domain_output[col_num].astype(float)
    domain_output[0] = domain_output[0].astype(str)
    display(domain_output)
    
    gnomad_iwes_overlap = pd.merge(domain_output, gnomad, on = [0, 1,2, 3, 4], how = "inner")
    return gnomad_iwes_overlap

def return_iWES_minus_gnomad_counts(region_type, ENST):
    if region_type == "cds":
        gnomad_iwes_overlap = return_iwes_gnomad_overlap("cds", ENST)
        return len(gnomad_iwes_overlap)
    else:
        gnomad_iwes_overlap = return_iwes_gnomad_overlap("domains", ENST)
        AD_counts = len(gnomad_iwes_overlap[gnomad_iwes_overlap["domain_type"] == "AD"])
        DBD_counts = len(gnomad_iwes_overlap[gnomad_iwes_overlap["domain_type"] == "DBD"])
        return (AD_counts, DBD_counts)

In [24]:
# Thresholding at 10th, 20th, ... 100th percentile allele frequency
threshold_percentiles = np.arange(0, 1.1, 0.1)
thresholds = [gnomad["gnomad_allele_freq"].quantile(percentile) for percentile in threshold_percentiles]
thresholds

[6.840441235821475e-07,
 7.034588791725314e-07,
 1.2003216862e-06,
 1.39551214454e-06,
 1.59195312264e-06,
 2.0521969452e-06,
 3.1051665337400044e-06,
 4.8013097973e-06,
 6.5758325003e-06,
 1.4319549338910008e-05,
 0.99641397952794]

In [ ]:
# 